# Apriori Association

## Importing the libraries

In [1]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=4d3fc8412a9d379790a8820c79f77aa3bce299b2632d77cfd62044266a7adcf8
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Preprocessing

In [13]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None) # None: no column names
row_count = len(dataset.index)
col_count = len(dataset.columns)
transactions = [] # list of lists of strings
for i in range(0, row_count):
  transactions.append([str(dataset.values[i,j]) for j in range(0, col_count)])
  # when a value doesn't exist it will be filled with 'nan'

In [18]:
print(transactions[327]) # example of a transaction (#327)

['red wine', 'mineral water', 'soup', 'rice', 'cookies', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']


## Training the Apriori model on the dataset

In [20]:
from apyori import apriori
rules = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)
# min_support: look for rules repeated 3 times per day --> 3 * 7 (a week) / 7501 (in a week) = 0.003 (0.3%)

## Visualising the results

### Displaying the first results coming directly from the output of the apriori function

In [22]:
results = list(rules)

In [25]:
results

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

In [41]:
result = results[0] # example result
print('Left Hand Side of the Rule:', result[2][0][0])
print('Right Hand Side of the Rule:', result[2][0][1])

Left Hand Side of the Rule: frozenset({'light cream'})
Right Hand Side of the Rule: frozenset({'chicken'})


### Putting the results well organised into a Pandas DataFrame

In [44]:
def inspect(results):
    left        = [tuple(result[2][0][0])[0] for result in results]
    right       = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(left, right, supports, confidences, lifts))
columnLabels = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift']
resultsinDataFrame = pd.DataFrame(inspect(results), columns = columnLabels)

### Displaying the results non sorted

In [45]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


### Displaying the results sorted by descending lifts

In [46]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710
